In [10]:
import numpy as np


# im2col for dense matrix
# creator function
def im2col_matrix_dense(Xin, K, S=1):
    N, Cin, Hin, Win = Xin.shape
    Hout, Wout = (Hin - K)//S + 1, (Win - K)//S + 1 # need to change this
    P = Hout * Wout  # Total number of patches per image
    patch_size = Cin * K * K # Size of each flattened patch
    im2col_mat_dense = np.zeros((Cin * Hin * Win, P * patch_size))
    patch_idx = 0
    for hout in range(Hout):
        for wout in range(Wout):
            for cin in range(Cin):
                for hker in range(K):
                    for wker in range(K):
                        input_index = cin * Hin * Win + hout * Win + wout + hker * Win + wker
                        patch_position = cin * K * K + hker * K + wker
                        output_index = patch_idx * patch_size + patch_position
                        im2col_mat_dense[input_index, output_index] = 1
            patch_idx += 1
    return im2col_mat_dense


In [11]:
Xin = np.arange(2*3*5*5).reshape(2,3,5,5)
print("Showing the first image, i.e., X[0]")
Xin[0]

Showing the first image, i.e., X[0]


array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24]],

       [[25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44],
        [45, 46, 47, 48, 49]],

       [[50, 51, 52, 53, 54],
        [55, 56, 57, 58, 59],
        [60, 61, 62, 63, 64],
        [65, 66, 67, 68, 69],
        [70, 71, 72, 73, 74]]])

In [12]:
N, Cin, Hin, Win = Xin.shape

K = 2
S = 1

Hout, Wout = Hin - K + 1, Win - K + 1
P = Hout * Wout

patch_size = Cin * K * K

Xin_flat = Xin.reshape(-1, Cin * Hin * Win)

im2col_mat = im2col_matrix_dense(Xin, K,S) # im2col

Xin_im2col = Xin_flat @ im2col_mat

Xin_patches_flat = Xin_im2col.reshape(N, P, patch_size) # X_out has shape (N, P, C*K**2)
Xin_patches = Xin_patches_flat.reshape(N, P, Cin, K, K)

print("Showing the first two patches of X[0]")
Xin_patches[0][:2]

Showing the first two patches of X[0]


array([[[[ 0.,  1.],
         [ 5.,  6.]],

        [[25., 26.],
         [30., 31.]],

        [[50., 51.],
         [55., 56.]]],


       [[[ 1.,  2.],
         [ 6.,  7.]],

        [[26., 27.],
         [31., 32.]],

        [[51., 52.],
         [56., 57.]]]])

## Apply a (flattened) convolutional filter

In [13]:
Cout = 2
weight_flat = np.array([[ 0.,  0.], # the convolutional filter
       [ 1.,  0.],
       [-1.,  0.],
       [ 0.,  0.],
       [ 0., -1.],
       [ 0.,  0.],
       [ 0.,  1.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])

Xout_flat = Xin_patches_flat @ weight_flat
Xout = np.moveaxis(Xout_flat, 1,2).reshape(N,Cout,Hout,Wout)

Xout

array([[[[-4., -4., -4., -4.],
         [-4., -4., -4., -4.],
         [-4., -4., -4., -4.],
         [-4., -4., -4., -4.]],

        [[ 5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.]]],


       [[[-4., -4., -4., -4.],
         [-4., -4., -4., -4.],
         [-4., -4., -4., -4.],
         [-4., -4., -4., -4.]],

        [[ 5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.]]]])

# Exercise 1: Modify `im2col_matrix_dense` to allow stride (note that kernel size has changed)

In [14]:
N, Cin, Hin, Win = Xin.shape

K = 3
S = 2

Hout, Wout = (Hin - K)//S + 1,  (Win - K)//S + 1

P = Hout * Wout

patch_size = Cin * K * K

Xin_flat = Xin.reshape(-1, Cin * Hin * Win)

im2col_mat = im2col_matrix_dense(Xin, K,S) # im2col

Xin_im2col = Xin_flat @ im2col_mat

Xin_patches_flat = Xin_im2col.reshape(N, P, patch_size) # X_out has shape (N, P, C*K**2)
Xin_patches = Xin_patches_flat.reshape(N, P, Cin, K, K)



print("Showing the first two patches of X[0]")
Xin_patches[0][:2]

Showing the first two patches of X[0]


array([[[[ 0.,  1.,  2.],
         [ 5.,  6.,  7.],
         [10., 11., 12.]],

        [[25., 26., 27.],
         [30., 31., 32.],
         [35., 36., 37.]],

        [[50., 51., 52.],
         [55., 56., 57.],
         [60., 61., 62.]]],


       [[[ 1.,  2.,  3.],
         [ 6.,  7.,  8.],
         [11., 12., 13.]],

        [[26., 27., 28.],
         [31., 32., 33.],
         [36., 37., 38.]],

        [[51., 52., 53.],
         [56., 57., 58.],
         [61., 62., 63.]]]])

# Exercise 1: Answer key:

In [4]:
answer = np.array([[[[ 0.,  1.,  2.],
         [ 5.,  6.,  7.],
         [10., 11., 12.]],

        [[25., 26., 27.],
         [30., 31., 32.],
         [35., 36., 37.]],

        [[50., 51., 52.],
         [55., 56., 57.],
         [60., 61., 62.]]],


       [[[ 2.,  3.,  4.],
         [ 7.,  8.,  9.],
         [12., 13., 14.]],

        [[27., 28., 29.],
         [32., 33., 34.],
         [37., 38., 39.]],

        [[52., 53., 54.],
         [57., 58., 59.],
         [62., 63., 64.]]]])


answer

array([[[[ 0.,  1.,  2.],
         [ 5.,  6.,  7.],
         [10., 11., 12.]],

        [[25., 26., 27.],
         [30., 31., 32.],
         [35., 36., 37.]],

        [[50., 51., 52.],
         [55., 56., 57.],
         [60., 61., 62.]]],


       [[[ 2.,  3.,  4.],
         [ 7.,  8.,  9.],
         [12., 13., 14.]],

        [[27., 28., 29.],
         [32., 33., 34.],
         [37., 38., 39.]],

        [[52., 53., 54.],
         [57., 58., 59.],
         [62., 63., 64.]]]])